In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import sam_stuff

## Get City data

### Step 1 - Get the data

In [9]:
# Create an empty DataFrame with specified columns to store city data:
city_df = pd.DataFrame(columns = ["city","country","population","latitude","longitude"]) 

def citygrab2(city):
    # Build the Wikipedia URL for the given city
    url = "https://en.wikipedia.org/wiki/"+city
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    # Parse the HTML content of the page with BeautifulSoup
    soup_city = BeautifulSoup(response.content, 'html.parser')
    
    # Find the population data in the infobox
    for header in soup_city.find_all(class_="infobox-header"):
        if header.find(string=re.compile("population", re.IGNORECASE)):
            popu = header.find_next(class_="infobox-data").text
    
    # Find the country data in the infobox        
    for merger in soup_city.find_all(class_="mergedtoprow"):
        if merger.find(string=re.compile("country", re.IGNORECASE)):
            country = merger.find_next(class_="infobox-data").text
    
    # Find the country data for different page structure (e.g., London)        
    for merger in soup_city.find_all(class_="mergedrow"):    
        if merger.find(string=re.compile("country", re.IGNORECASE)):
            country = merger.find_next(class_="infobox-data").text
    
    # Find the coordinates data in the infobox
    for coords in soup_city.find_all(class_="infobox-full-data"):
        if coords.find(string=re.compile("Coordinates", re.IGNORECASE)):
            lati = coords.find_next(class_="latitude").text
            longi = coords.find_next(class_="longitude").text
   
    # Add the extracted data to the DataFrame        
    city_df.loc[len(city_df)]=({"city":str.capitalize(city), "country":country,"population":popu,"latitude":lati,"longitude":longi})
   
    return city_df

# Example usage: Fetch and add data for Berlin, and London
citygrab2("Berlin")
city = citygrab2("London")
# Clean up the population data: remove references and commas, convert to numeric
city['population'] = city['population'].str.replace('[1]','').str.replace(',','')
city["population"] = pd.to_numeric(city["population"])
# Display the DataFrame
city

,city,country,population,latitude,longitude
0,Berlin,Germany,3755251,52°31′12″N,13°24′18″E
1,London,England,8799800,51°30′26″N,0°7′39″W


### Step 2 - create a conection string

In [21]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password = sam_stuff.my_sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### Step 3 - Send data to mySQL

In [ ]:
city.to_sql(name='city',if_exists='append',con=connection_string,index=False)

## Get the airport data

In [ ]:
#Fetch the city data from the database and load it into a DataFrame
cities_df = pd.read_sql('city',con=connection_string)

# Create a dictionary to store airport data
airports_dict = {'ICAO':[],'city_id':[]}

# Define a dictionary mapping city names to their respective ICAO codes
icao_dict = { 'Berlin':'EDDB',
             'London':'EGLL' }

# Iterate through the DataFrame to populate the airports_dict and append the city_id and ICAO to the dictionary
for i, row in cities_df.iterrows():
    airports_dict['city_id'].append(row['city_id'])  
    airports_dict['ICAO'].append(icao_dict[row['city']])

# Convert the dictionary into a DataFrame:
airports_df = pd.DataFrame(airports_dict)

# Display resulting DataFrame:
airports_df

### Send data to SQL

In [ ]:
#airports_df.to_sql('airport',if_exists='append',index=False, con=connection_string)